In [ ]:
# import libraries
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive

In [ ]:
bias_and_weights = []

#fake fourth weight for example
new_weights = np.array([3.55366, 2.01605, -1.19409, -2.47392, -1.45932, 0.28493, 0.38204,0])
new_weights = new_weights.reshape(8,1)
bias = np.array([0]).reshape(1,)

bias_and_weights.append(new_weights)
bias_and_weights.append(bias)

print("wanted shape:", new_weights.shape)

model = Sequential()
model.add(Flatten(input_shape = new_weights.shape))
model.add(Dense(1, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy")

print(model.get_weights())

model.set_weights(bias_and_weights)
print(model.get_weights())

model.intercept = -3.57712
model.summary()

print("numer of tensors: ", len(model.inputs))

wanted shape: (8, 1)
[array([[ 0.32426178],
       [ 0.6161803 ],
       [ 0.23147905],
       [ 0.34872508],
       [-0.78231525],
       [-0.3218535 ],
       [-0.09668458],
       [-0.7337028 ]], dtype=float32), array([0.], dtype=float32)]
[array([[ 3.55366],
       [ 2.01605],
       [-1.19409],
       [-2.47392],
       [-1.45932],
       [ 0.28493],
       [ 0.38204],
       [ 0.     ]], dtype=float32), array([0.], dtype=float32)]
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________
numer of tensor

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('ebola4Features.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
!pip install tflite-support-nightly

In [ ]:
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

""" ... """
"""Creates the metadata for an image classifier."""

# Creates model info.
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "EbolaDetection"
model_meta.description = ("This model diagnoses whether a patient has Ebola or not depending on the variables bleeding (anywhere on the person), previous contact with an indvidual that has Ebola, and abdominal pain.")
model_meta.version = "v1"
model_meta.author = "Colabo Lab"

# Creates input info.
input_meta1 = _metadata_fb.TensorMetadataT()

# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()

# feature
input_meta1.name = "EbolaContact, Bleeding, AbdominalPain, Transport"
input_meta1.description = (
    "Feature #1 - Provides information about whether a patient has been in contact with another individual diagnosed with Ebola.\n Feature #2 - Provides information about any bleeding the patient may be experiencing. \n Feature #3 - Provides any information about abdominal pain the patient may be experiencing\nFeature #4 - Provides any information about transportation to care.\n".format(160, 160))
input_meta1.content = _metadata_fb.ContentT()
input_meta1.content.content_properties = _metadata_fb.FeaturePropertiesT()
input_meta1.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)

# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "probability"
output_meta.description = "Probabilty of having contracted Ebola."
output_meta.content = _metadata_fb.ContentT()
output_meta.content.content_properties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)

# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta1]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

populator = _metadata.MetadataPopulator.with_model_file("/content/ebola4Features.tflite")
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files(["/content/ebola4Features.tflite"])
populator.populate()

/usr/local/lib/python3.10/dist-packages/tensorflow_lite_support/metadata/python/metadata.py:395: UserWarning: File, 'ebola4Features.tflite', does not exist in the metadata. But packing it to tflite model is still allowed.
  warnings.warn(
